In [ ]:
#connect to google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import numpy as np
# Load the data from Google Drive
y = np.load('/content/gdrive/MyDrive/ECG/arr2D_AF&SR_label.npy')
X = np.load('/content/gdrive/MyDrive/ECG/items_AF&SR.npy')

In [ ]:
import numpy as np

# Assuming arr is your (6428, 5000, 12) shaped array
arr = X
# Indices to remove from the third dimension
indices_to_remove = [2]  # 1st, 3rd, 7th, 8th positions

# Remove the specified positions from the third dimension
arr = np.delete(arr, indices_to_remove, axis=2)

# Now, the shape of arr will be (6428, 5000, 8)
print(arr.shape)

(3587, 1000, 2)


In [ ]:
print(X.shape)
print(X)

(3587, 1000, 3)
[[[ 0.135 -0.125 -0.08 ]
  [ 0.135 -0.125 -0.08 ]
  [ 0.135 -0.125 -0.08 ]
  ...
  [-0.125 -0.066 -0.07 ]
  [-0.126 -0.063 -0.07 ]
  [-0.124 -0.06  -0.07 ]]

 [[-0.13  -0.045 -0.03 ]
  [-0.13  -0.045 -0.03 ]
  [-0.13  -0.045 -0.03 ]
  ...
  [ 0.003  0.027 -0.02 ]
  [ 0.005  0.025 -0.018]
  [ 0.005  0.024 -0.015]]

 [[-0.47  -0.43  -0.72 ]
  [-0.47  -0.43  -0.72 ]
  [-0.47  -0.43  -0.72 ]
  ...
  [-0.064  0.111  0.471]
  [-0.075  0.11   0.467]
  [-0.081  0.11   0.458]]

 ...

 [[ 0.025 -0.065  0.06 ]
  [ 0.025 -0.065  0.06 ]
  [ 0.025 -0.065  0.06 ]
  ...
  [-0.012 -0.035 -0.018]
  [-0.018 -0.006 -0.016]
  [-0.02   0.013 -0.021]]

 [[ 0.15   0.15   0.02 ]
  [ 0.15   0.15   0.02 ]
  [ 0.15   0.15   0.02 ]
  ...
  [ 0.121  0.05   0.063]
  [ 0.146  0.049  0.067]
  [ 0.166  0.052  0.071]]

 [[ 0.015  0.095 -0.005]
  [ 0.015  0.095 -0.005]
  [ 0.015  0.095 -0.005]
  ...
  [-0.102  0.113 -0.13 ]
  [-0.099  0.124 -0.129]
  [-0.1    0.125 -0.124]]]


In [ ]:
# Preprocessing
import numpy as np
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)

print(X_scaled)

[[[ 0.80705828 -0.4588596  -0.25228113]
  [ 0.80705828 -0.4588596  -0.25228113]
  [ 0.80705828 -0.4588596  -0.25228113]
  ...
  [-0.71186622 -0.24757386 -0.22215566]
  [-0.71770824 -0.23683052 -0.22215566]
  [-0.7060242  -0.22608717 -0.22215566]]

 [[-0.7410763  -0.17237046 -0.10165378]
  [-0.7410763  -0.17237046 -0.10165378]
  [-0.7410763  -0.17237046 -0.10165378]
  ...
  [ 0.035912    0.08546976 -0.07152831]
  [ 0.04759603  0.07830753 -0.06550321]
  [ 0.04759603  0.07472642 -0.05646557]]

 [[-2.72736219 -1.55109943 -2.18031125]
  [-2.72736219 -1.55109943 -2.18031125]
  [-2.72736219 -1.55109943 -2.18031125]
  ...
  [-0.35550316  0.38628336  1.4076323 ]
  [-0.41976535  0.38270224  1.39558212]
  [-0.45481746  0.38270224  1.36846919]]

 ...

 [[ 0.16443638 -0.24399274  0.16947546]
  [ 0.16443638 -0.24399274  0.16947546]
  [ 0.16443638 -0.24399274  0.16947546]
  ...
  [-0.05171826 -0.13655932 -0.06550321]
  [-0.08677037 -0.03270701 -0.05947812]
  [-0.0984544   0.03533416 -0.07454085]]

 [

In [ ]:
import numpy as np

# Assuming X and Y are your arrays
X_s = X_scaled
Y_s = y
# Generate indices for splitting
indices = np.arange(len(Y_s))

# Split the indices into train and test sets
test_indices = indices[::5]  # Every fifth index
train_indices = np.setdiff1d(indices, test_indices)

# Split X and Y using the generated indices
X_tr = X_s[train_indices]
Y_tr = Y_s[train_indices]

X_ts = X_s[test_indices]
Y_ts = Y_s[test_indices]

# Check the shapes
print("X_train shape:", X_tr.shape)
print("Y_train shape:", Y_tr.shape)
print("X_test shape:", X_ts.shape)
print("Y_test shape:", Y_ts.shape)

X_train shape: (2869, 1000, 3)
Y_train shape: (2869, 1)
X_test shape: (718, 1000, 3)
Y_test shape: (718, 1)


In [ ]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Flatten, Bidirectional

# Define model parameters
num_classes = 2
input_shape = (1000, 3)  # assuming 1000 time steps and 3 features

# Reshape the data to fit the CNN input shape
X_train_cnn = X_tr.reshape((-1, input_shape[0], input_shape[1]))  # reshape without channel dimension
X_test_cnn = X_ts.reshape((-1, input_shape[0], input_shape[1]))

# Define CNN-Bi-LSTM model architecture
model = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', input_shape=input_shape),
    MaxPooling1D(pool_size=2),
    Conv1D(128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Bidirectional(LSTM(64, return_sequences=True)),
    Bidirectional(LSTM(32)),
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_8 (Conv1D)           (None, 998, 64)           640       
                                                                 
 max_pooling1d_8 (MaxPoolin  (None, 499, 64)           0         
 g1D)                                                            
                                                                 
 conv1d_9 (Conv1D)           (None, 497, 128)          24704     
                                                                 
 max_pooling1d_9 (MaxPoolin  (None, 248, 128)          0         
 g1D)                                                            
                                                                 
 bidirectional_8 (Bidirecti  (None, 248, 128)          98816     
 onal)                                                           
                                                      

In [ ]:
# Train the model
model.fit(X_train_cnn, Y_tr, epochs=100, batch_size=64, validation_split=0.1)

# Evaluate the model on test data
loss, accuracy = model.evaluate(X_test_cnn, Y_ts)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

In [ ]:
# Define the light LSTM student model
def create_student_model(input_shape, num_classes):
    model = Sequential([
        LSTM(32, return_sequences=True, input_shape=input_shape),
        LSTM(32),
        Dense(64, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    return model

student_model = create_student_model(input_shape, num_classes)
student_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
student_model.summary()

student_model.fit(X_train_cnn, Y_tr, epochs=100, batch_size=64, validation_split=0.1)

# Evaluate the model on test data
loss, accuracy = student_model.evaluate(X_ts, Y_ts)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)



Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_16 (LSTM)              (None, 1000, 32)          4608      
                                                                 
 lstm_17 (LSTM)              (None, 32)                8320      
                                                                 
 dense_18 (Dense)            (None, 64)                2112      
                                                                 
 dense_19 (Dense)            (None, 2)                 130       
                                                                 
Total params: 15170 (59.26 KB)
Trainable params: 15170 (59.26 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
import tensorflow as tf
from keras import Model
from keras.losses import KLDivergence

# Custom training step for knowledge distillation
class DistillationModel(Model):
    def __init__(self, student, teacher, alpha=0.1, temperature=3):
        super(DistillationModel, self).__init__()
        self.student = student
        self.teacher = teacher
        self.alpha = alpha
        self.temperature = temperature
        self.student_loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
        self.distillation_loss_fn = KLDivergence()

    def compile(self, optimizer, metrics):
        super(DistillationModel, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
        self.distillation_loss_fn = KLDivergence()

    def train_step(self, data):
        x, y = data
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            student_predictions = self.student(x, training=True)
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1)
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss * (self.temperature ** 2)

        gradients = tape.gradient(loss, self.student.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.student.trainable_variables))

        self.compiled_metrics.update_state(y, student_predictions)
        return {m.name: m.result() for m in self.metrics}

    def test_step(self, data):
        x, y = data
        y_pred = self.student(x, training=False)
        loss = self.student_loss_fn(y, y_pred)

        self.compiled_metrics.update_state(y, y_pred)
        return {m.name: m.result() for m in self.metrics}

# Instantiate and compile the distillation model
distillation_model = DistillationModel(student=student_model, teacher=model, alpha=0.1, temperature=2)
distillation_model.compile(optimizer='adam', metrics=['accuracy'])

# Train the student model with knowledge distillation
distillation_model.fit(X_train_cnn, Y_tr, epochs=100, batch_size=64, validation_split=0.1)


Epoch 1/100
41/41 [==============================] - 12s 120ms/step - accuracy: 0.8331 - val_accuracy: 0.7422
Epoch 2/100
41/41 [==============================] - 3s 73ms/step - accuracy: 0.8435 - val_accuracy: 0.7526
Epoch 3/100
41/41 [==============================] - 3s 81ms/step - accuracy: 0.8389 - val_accuracy: 0.7491
Epoch 4/100
41/41 [==============================] - 4s 95ms/step - accuracy: 0.7804 - val_accuracy: 0.7317
Epoch 5/100
41/41 [==============================] - 3s 74ms/step - accuracy: 0.7796 - val_accuracy: 0.7317
Epoch 6/100
41/41 [==============================] - 3s 74ms/step - accuracy: 0.7730 - val_accuracy: 0.7561
Epoch 7/100
41/41 [==============================] - 3s 73ms/step - accuracy: 0.8187 - val_accuracy: 0.7317
Epoch 8/100
41/41 [==============================] - 4s 103ms/step - accuracy: 0.8350 - val_accuracy: 0.7387
Epoch 9/100
41/41 [==============================] - 3s 75ms/step - accuracy: 0.8296 - val_accuracy: 0.7282
Epoch 10/100
41/41 [=====